In [1]:
# =====================================================
# 0️⃣ 라이브러리
# =====================================================
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
    classification_report,
    roc_auc_score,
    average_precision_score
)

import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Embedding, Dense, Concatenate,
    Flatten, Dropout
)
from tensorflow.keras.models import Model

In [2]:
# =====================================================
# 1️⃣ 데이터 로드
# =====================================================
csv_path = "flight_status_final_filtered.csv"
df = pd.read_csv(csv_path)

print("✅ CSV 로드 완료")
print("행 수:", len(df))

C:\Users\Admin\AppData\Local\Temp\ipykernel_9024\374033911.py:5: DtypeWarning: Columns (6,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


✅ CSV 로드 완료
행 수: 1500000


In [3]:
# =====================================================
# 2️⃣ 타겟 변수 생성
# =====================================================
def make_delay_label(x):
    if pd.isna(x):
        return 0
    if any(k in x for k in ["지연", "결항", "회항"]):
        return 1
    return 0

df["is_delay"] = df["RMK_KOR"].apply(make_delay_label)
print(df["is_delay"].value_counts())

0    1487392
1      12608
Name: is_delay, dtype: int64


In [4]:
# =====================================================
# 3️⃣ 날짜 / 시간 파생 (weekday 유지)
# =====================================================
df["flight_date"] = pd.to_datetime(df["FLIGHT_DATE"], format="%Y%m%d")

# 요일 (0=월, 6=일)
df["weekday"] = df["flight_date"].dt.weekday

# 월
df["month"] = df["flight_date"].dt.month

# 출발 시간
df["STD"] = df["STD"].astype(str)
df["hour"] = df["STD"].str.zfill(4).str[:2].astype(int)



In [5]:
# =====================================================
# 4️⃣ Train / Test 분리 (날짜 기준)
# =====================================================
df = df.sort_values("flight_date")
split_date = df["flight_date"].quantile(0.8)

train_df = df[df["flight_date"] <= split_date]
test_df = df[df["flight_date"] > split_date]

print("Train:", len(train_df), "Test:", len(test_df))

Train: 1200582 Test: 299418


In [8]:
from sklearn.preprocessing import LabelEncoder

embed_cols = [
    "AIRLINE_KOREAN",
    "BOARDING_KOR",
    "ARRIVED_KOR",
    "LINE",
    "IO"
]


label_encoders = {}

for col in embed_cols:
    # 복사본으로 작업 (경고 제거)
    train_df = train_df.copy()
    test_df = test_df.copy()

    # 문자열 처리
    train_df.loc[:, col] = train_df[col].astype(str)
    test_df.loc[:, col] = test_df[col].astype(str)

    # train 기준으로 클래스 정의
    le = LabelEncoder()
    le.fit(train_df[col])

    # train 변환
    train_df.loc[:, col] = le.transform(train_df[col])

    # test에서 unseen label → 'UNKNOWN'
    test_df.loc[:, col] = test_df[col].apply(
        lambda x: x if x in le.classes_ else "UNKNOWN"
    )

    # UNKNOWN을 classes_에 추가
    le.classes_ = np.append(le.classes_, "UNKNOWN")

    # test 변환
    test_df.loc[:, col] = le.transform(test_df[col])

    label_encoders[col] = le


C:\Users\Admin\AppData\Local\Temp\ipykernel_9024\4290059788.py:28: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  train_df.loc[:, col] = le.transform(train_df[col])
C:\Users\Admin\AppData\Local\Temp\ipykernel_9024\4290059788.py:39: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  test_df.loc[:, col] = le.transform(test_df[col])
C:\Users\Admin\AppData\Local\Temp\ipykernel_9024\4290059788.py:28: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retai

In [9]:
# =====================================================
# 6️⃣ 입력 데이터 구성
# =====================================================
num_cols = ["weekday", "month", "hour"]

X_train_embed = train_df[embed_cols].values
X_test_embed = test_df[embed_cols].values

X_train_num = train_df[num_cols].values
X_test_num = test_df[num_cols].values

y_train = train_df["is_delay"].values
y_test = test_df["is_delay"].values

In [10]:
# =====================================================
# 7️⃣ 임베딩 + MLP 모델 정의
# =====================================================
inputs = []
embeddings = []

for col in embed_cols:
    input_dim = df[col].nunique() + 1
    emb_dim = min(8, input_dim // 2)

    inp = Input(shape=(1,), name=f"{col}_input")
    emb = Embedding(
        input_dim=input_dim,
        output_dim=emb_dim,
        name=f"{col}_emb"
    )(inp)
    emb = Flatten()(emb)

    inputs.append(inp)
    embeddings.append(emb)

# 수치형 입력
num_input = Input(shape=(len(num_cols),), name="num_input")
inputs.append(num_input)

# 결합
x = Concatenate()(embeddings + [num_input])

# MLP 본체
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)
x = Dense(64, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(32, activation="relu")(x)

output = Dense(1, activation="sigmoid")(x)

model = Model(inputs=inputs, outputs=output)

In [11]:

# =====================================================
# 8️⃣ 컴파일
# =====================================================
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=[
        tf.keras.metrics.AUC(name="roc_auc"),
        tf.keras.metrics.AUC(curve="PR", name="pr_auc")
    ]
)

model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 AIRLINE_KOREAN_input (InputLay  [(None, 1)]         0           []                               
 er)                                                                                              
                                                                                                  
 BOARDING_KOR_input (InputLayer  [(None, 1)]         0           []                               
 )                                                                                                
                                                                                                  
 ARRIVED_KOR_input (InputLayer)  [(None, 1)]         0           []                               
                                                                                              

In [12]:
# =====================================================
# 9️⃣ 학습
# =====================================================
history = model.fit(
    [
        X_train_embed[:, i] for i in range(len(embed_cols))
    ] + [X_train_num],
    y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=2048,
    class_weight={0: 1, 1: 70},
    verbose=1
)


Epoch 1/10
469/469 [==============================] - 5s 7ms/step - loss: 1.0187 - roc_auc: 0.6417 - pr_auc: 0.0175 - val_loss: 0.5162 - val_roc_auc: 0.7162 - val_pr_auc: 0.0298
Epoch 2/10
469/469 [==============================] - 3s 7ms/step - loss: 0.9154 - roc_auc: 0.7458 - pr_auc: 0.0318 - val_loss: 0.4658 - val_roc_auc: 0.7244 - val_pr_auc: 0.0308
Epoch 3/10
469/469 [==============================] - 3s 7ms/step - loss: 0.8736 - roc_auc: 0.7772 - pr_auc: 0.0396 - val_loss: 0.4227 - val_roc_auc: 0.7174 - val_pr_auc: 0.0282
Epoch 4/10
469/469 [==============================] - 4s 8ms/step - loss: 0.8496 - roc_auc: 0.7934 - pr_auc: 0.0424 - val_loss: 0.4723 - val_roc_auc: 0.7275 - val_pr_auc: 0.0301
Epoch 5/10
469/469 [==============================] - 3s 7ms/step - loss: 0.8314 - roc_auc: 0.8034 - pr_auc: 0.0480 - val_loss: 0.4345 - val_roc_auc: 0.7039 - val_pr_auc: 0.0269
Epoch 6/10
469/469 [==============================] - 3s 7ms/step - loss: 0.8141 - roc_auc: 0.8137 - pr_auc: 0

In [13]:
# =====================================================
# 🔟 평가
# =====================================================
y_prob = model.predict(
    [
        X_test_embed[:, i] for i in range(len(embed_cols))
    ] + [X_test_num]
).ravel()

print("ROC-AUC:", roc_auc_score(y_test, y_prob))
print("PR-AUC:", average_precision_score(y_test, y_prob))

for t in [0.3, 0.4, 0.5, 0.6]:
    y_pred_t = (y_prob >= t).astype(int)
    print(f"\n===== Threshold = {t} =====")
    print(classification_report(y_test, y_pred_t))

9357/9357 [==============================] - 8s 803us/step
ROC-AUC: 0.6889385814504434
PR-AUC: 0.008918972713924962

===== Threshold = 0.3 =====
              precision    recall  f1-score   support

           0       1.00      0.59      0.74    298339
           1       0.01      0.66      0.01      1079

    accuracy                           0.59    299418
   macro avg       0.50      0.62      0.38    299418
weighted avg       0.99      0.59      0.74    299418


===== Threshold = 0.4 =====
              precision    recall  f1-score   support

           0       1.00      0.68      0.81    298339
           1       0.01      0.57      0.01      1079

    accuracy                           0.68    299418
   macro avg       0.50      0.62      0.41    299418
weighted avg       0.99      0.68      0.81    299418


===== Threshold = 0.5 =====
              precision    recall  f1-score   support

           0       1.00      0.75      0.85    298339
           1       0.01      0.53 